# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-3cae106501-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Cultural Movements explorative search") 

Consider the following exploratory information need:

> investigate the cultural movements, in particular the Renaissance, Baroque and Romanticsm. We'd like to get an overview of the main artists belonging to these movements, musicians, poets, sculptors and painters.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q37562`| Donatello    | node |
| `wd:Q19675`| Louvre Museum | node |





Also consider

```
wd:Q37562 ?p ?obj .
```

is the BGP to retrieve all **properties of Donatello**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Cultural Movement

2. Identify the BGP for Renassaince, Baroque and Romanticism

3. Identify the BGP for composer, painter, sculptor and poet

4. Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

5. Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

6. For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

7. Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

## Task 1
Identify the BGP for Cultural Movement

In [2]:
# I explore the node donatello to see what properties has. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/dir

With this result, I am going to try and see what properties has 

In [29]:
# I explore the node, human, to see his properties. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q5 ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P129'), ('name', 'physically interacts with')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('name', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('name', 'studied by')]
[('p

In [32]:
# I am going to look for the properties of movement. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:P135 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('name', 'see also')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('name', 'Wikidata item of this property')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('name', 'Wikidata property example')]
[('p', 'http://www.wikidata.org/prop/direct/P2302'), ('name', 'property constraint')]
[('p', 'http://www.wikidata.org/prop/direct/P2875'), ('name', 'property usage tracking category')]
[('p', 'http://www.wikidata.org/prop/direct/P3734'), ('name', 'category for value same as Wikidata')]
[('p', 'http://www.wikidata.org/prop/direct/P7087'), ('name', 'inverse label item')]
8


In [31]:
# I explore movements on Donatello to see the values for him. 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   ?obj wdt:P135 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q1472236'), ('iname', 'Early Renaissance')]
[('inst', 'http://www.wikidata.org/entity/Q1474884'), ('iname', 'High Renaissance')]
2


In [34]:
# I look for the values of high renaissance, one of the values from the previus query. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1474884 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/pro

In [11]:
# On this query, I am going to look from who it's instance High Renaissance 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q1474884 ?p ?obj .
   ?obj wdt:P31 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
GROUP BY ?inst ?iname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q2198855'), ('iname', 'cultural movement')]
[('inst', 'http://www.wikidata.org/entity/Q32880'), ('iname', 'architectural style')]
[('inst', 'http://www.wikidata.org/entity/Q968159'), ('iname', 'art movement')]
[('inst', 'http://www.wikidata.org/entity/Q19361238'), ('iname', 'Wikidata metaclass')]
[('inst', 'http://www.wikidata.org/entity/Q4167836'), ('iname', 'Wikimedia category')]
[('inst', 'http://www.wikidata.org/entity/Q11514315'), ('iname', 'historical period')]
[('inst', 'http://www.wikidata.org/entity/Q1292119'), ('iname', 'style')]
[('inst', 'http://www.wikidata.org/entity/Q3533467'), ('iname', 'group action')]
[('inst', 'http://www.wikidata.org/entity/Q151885'), ('iname', 'concept')]
9


I found cultural movement so that is the final query. 

Final query for this task

In [20]:
# write your final query 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q1474884 ?p ?obj .
   ?obj wdt:P31 ?inst .
   
   #I use "values" to only show the value from cultural movement wich I've got from the previous query. 
   VALUES ?inst { wd:Q2198855 } 
   
   # get the label
   ?inst sc:name ?iname.
}
GROUP BY ?inst ?iname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q2198855'), ('iname', 'cultural movement')]
1


## Task 2
Identify the BGP for Renassaince, Baroque and Romanticism

In [18]:
# I explore the instances of Curtural Movement
queryString = """
SELECT ?item ?name 
WHERE {
  
   ?item wdt:P31 wd:Q2198855. #all items that hace instance of- with the value Q2198855 "cultural movement" 
   # get the label
   ?item sc:name ?name.
}

LIMIT 30

"""

print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q8210150'), ('name', 'Avanzada Regia')]
[('item', 'http://www.wikidata.org/entity/Q211481'), ('name', 'Dogme 95')]
[('item', 'http://www.wikidata.org/entity/Q2046861'), ('name', 'Palaeologan Renaissance')]
[('item', 'http://www.wikidata.org/entity/Q1400430'), ('name', 'Macedonian Renaissance')]
[('item', 'http://www.wikidata.org/entity/Q430167'), ('name', 'Spanish Renaissance')]
[('item', 'http://www.wikidata.org/entity/Q1814286'), ('name', 'Young Estonia')]
[('item', 'http://www.wikidata.org/entity/Q2614483'), ('name', 'Pre-romanticism')]
[('item', 'http://www.wikidata.org/entity/Q39427'), ('name', 'surrealism')]
[('item', 'http://www.wikidata.org/entity/Q2349908'), ('name', 'French Renaissance')]
[('item', 'http://www.wikidata.org/entity/Q878985'), ('name', 'modernism')]
[('item', 'http://www.wikidata.org/entity/Q466887'), ('name', 'Belle Époque')]
[('item', 'http://www.wikidata.org/entity/Q12539'), ('name', 'Age of Enlightenment')]
[

Final query for this task

I found out with the last query the 3 BGPs that I was looking for. So now I filter to only show the ones that we want. 

In [17]:
# I explore the instances of Curtural Movement
queryString = """
SELECT ?item ?name 
WHERE {
  
   ?item wdt:P31 wd:Q2198855. #all items that hace instance of- with the value Q2198855 "cultural movement" 
   
   VALUES ?item {
   wd:Q4692 #Renaissance
   wd:Q37853 #Baroque
   wd:Q37068 #Romanticism
   }
   # get the label
   ?item sc:name ?name.
}

LIMIT 30

"""

print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
[('item', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
[('item', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
3


## Task 3
Identify the BGP for composer, painter, sculptor and poet

In [32]:
#I start looking for the identifier for the property occupation in donatello 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/dir

In [33]:
#now that I have the value for the property of occupation, I look for the values for donatello
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   ?obj wdt:P106 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q1028181'), ('iname', 'painter')]
[('inst', 'http://www.wikidata.org/entity/Q1281618'), ('iname', 'sculptor')]
[('inst', 'http://www.wikidata.org/entity/Q1281618'), ('iname', 'sculptor')]
[('inst', 'http://www.wikidata.org/entity/Q1281618'), ('iname', 'sculptor')]
[('inst', 'http://www.wikidata.org/entity/Q42973'), ('iname', 'architect')]
[('inst', 'http://www.wikidata.org/entity/Q1708232'), ('iname', 'medalist')]
6


In [24]:
#I Look for all the properties of painter
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1028181 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'i

In [32]:
#Now I try to find all the instances of painter
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q1028181 ?p ?obj .
   ?obj wdt:P31 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
GROUP BY ?inst ?iname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q5'), ('iname', 'human')]
[('inst', 'http://www.wikidata.org/entity/Q19887878'), ('iname', 'Wikimedia infobox template')]
[('inst', 'http://www.wikidata.org/entity/Q1799072'), ('iname', 'method')]
[('inst', 'http://www.wikidata.org/entity/Q97584729'), ('iname', 'Wikidata property related to biographical dictionaries')]
[('inst', 'http://www.wikidata.org/entity/Q1792644'), ('iname', 'art style')]
[('inst', 'http://www.wikidata.org/entity/Q55653847'), ('iname', 'Wikidata property for authority control for artists')]
[('inst', 'http://www.wikidata.org/entity/Q19361238'), ('iname', 'Wikidata metaclass')]
[('inst', 'http://www.wikidata.org/entity/Q4167836'), ('iname', 'Wikimedia category')]
[('inst', 'http://www.wikidata.org/entity/Q24017414'), ('iname', 'second-order class')]
[('inst', 'http://www.wikidata.org/entity/Q4167410'), ('iname', 'Wikimedia disambiguation page')]
[('inst', 'http://www.wikidata.org/entity/Q11266439'), ('iname', 'Wik

In [25]:
#I'm going to look now the properties from sculpor
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1281618 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wik

In [26]:
#all the instances of sculpor
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q1281618 ?p ?obj .
   ?obj wdt:P31 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
GROUP BY ?inst ?iname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q59542487'), ('iname', 'Wikimedia set category')]
[('inst', 'http://www.wikidata.org/entity/Q19887878'), ('iname', 'Wikimedia infobox template')]
[('inst', 'http://www.wikidata.org/entity/Q1799072'), ('iname', 'method')]
[('inst', 'http://www.wikidata.org/entity/Q3331189'), ('iname', 'version, edition, or translation')]
[('inst', 'http://www.wikidata.org/entity/Q19361238'), ('iname', 'Wikidata metaclass')]
[('inst', 'http://www.wikidata.org/entity/Q24017414'), ('iname', 'second-order class')]
[('inst', 'http://www.wikidata.org/entity/Q66666236'), ('iname', 'Wikimedia list of persons by occupation')]
[('inst', 'http://www.wikidata.org/entity/Q11266439'), ('iname', 'Wikimedia template')]
[('inst', 'http://www.wikidata.org/entity/Q88789639'), ('iname', 'artistic profession')]
[('inst', 'http://www.wikidata.org/entity/Q56055944'), ('iname', 'art form')]
[('inst', 'http://www.wikidata.org/entity/Q30432511'), ('iname', 'metacategory in Wikime

In [28]:
# Now all the properties of architect 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q42973 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikid

In [29]:
#all the instaces of architect 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q42973 ?p ?obj .
   ?obj wdt:P31 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
GROUP BY ?inst ?iname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q5'), ('iname', 'human')]
[('inst', 'http://www.wikidata.org/entity/Q21070568'), ('iname', 'human who may be fictional')]
[('inst', 'http://www.wikidata.org/entity/Q28640'), ('iname', 'profession')]
[('inst', 'http://www.wikidata.org/entity/Q19887878'), ('iname', 'Wikimedia infobox template')]
[('inst', 'http://www.wikidata.org/entity/Q97584729'), ('iname', 'Wikidata property related to biographical dictionaries')]
[('inst', 'http://www.wikidata.org/entity/Q21745557'), ('iname', 'Wikidata property to identify organizations')]
[('inst', 'http://www.wikidata.org/entity/Q187588'), ('iname', 'social class')]
[('inst', 'http://www.wikidata.org/entity/Q17152639'), ('iname', 'thesaurus')]
[('inst', 'http://www.wikidata.org/entity/Q56216473'), ('iname', 'Wikidata property for authority control for architects')]
[('inst', 'http://www.wikidata.org/entity/Q11862829'), ('iname', 'academic discipline')]
[('inst', 'http://www.wikidata.org/entity/Q193

I see that that 3 occupations have one instance in common, artistic profession, so I am going to try and look for all the occupations form artistic profession. 

In [33]:
#Q108300140
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q108300140 ?p ?obj .
   ?obj wdt:P31 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}
GROUP BY ?inst ?iname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q1799072'), ('iname', 'method')]
[('inst', 'http://www.wikidata.org/entity/Q828962'), ('iname', 'job classification system')]
2


In [21]:
#All the instance from artistic profession 
queryString = """

SELECT DISTINCT ?occupations ?occupationsName 
WHERE {

   #bind something 
   ?occupations wdt:P31 wd:Q88789639 . 
    
   # get the label 
   ?occupations sc:name ?occupationsName . 
   
   
}
#ORDER BY ASC (?occupationsName)




"""

print("Results")
x=run_query(queryString)

Results
[('occupations', 'http://www.wikidata.org/entity/Q98552221'), ('occupationsName', 'plastic artist')]
[('occupations', 'http://www.wikidata.org/entity/Q3391743'), ('occupationsName', 'visual artist')]
[('occupations', 'http://www.wikidata.org/entity/Q280447'), ('occupationsName', 'Carnivalesque')]
[('occupations', 'http://www.wikidata.org/entity/Q108596703'), ('occupationsName', 'urushi artist')]
[('occupations', 'http://www.wikidata.org/entity/Q108605121'), ('occupationsName', 'dye artist')]
[('occupations', 'http://www.wikidata.org/entity/Q40749908'), ('occupationsName', 'bronze sculptor')]
[('occupations', 'http://www.wikidata.org/entity/Q106669359'), ('occupationsName', 'interdisciplinary artist')]
[('occupations', 'http://www.wikidata.org/entity/Q56299528'), ('occupationsName', 'bonsai artist')]
8


With the last queries didn't worked, I am going to try to find the occupations through the people that belong to the movements like donatello. and see if I can get it through this path. 

In [5]:
#I'll try to show all the people that has the property movement. 
queryString = """

SELECT DISTINCT ?artist ?artistName 
WHERE {

   #bind something 
   ?artist wdt:P31 wd:Q5 . #all instance of human 
   ?artist wdt:P135 ?m. #all that has the property movement.   
   # get the label 
   ?artist sc:name ?artistName . 
   
   
}
ORDER BY ASC (?artistName)
LIMIT 20 #I put the limit because there's gonna be a huge amount of people. 



"""

print("Results")
x=run_query(queryString)

Results
[('artist', 'http://www.wikidata.org/entity/Q12222083'), ('artistName', "'A'isha al-Ba'uniyya")]
[('artist', 'http://www.wikidata.org/entity/Q182800'), ('artistName', 'A. B. Shah')]
[('artist', 'http://www.wikidata.org/entity/Q278583'), ('artistName', 'A. C. Grayling')]
[('artist', 'http://www.wikidata.org/entity/Q399986'), ('artistName', 'A. D. Gordon')]
[('artist', 'http://www.wikidata.org/entity/Q55524030'), ('artistName', 'A. D. Painter')]
[('artist', 'http://www.wikidata.org/entity/Q3600312'), ('artistName', 'A. D. Winans')]
[('artist', 'http://www.wikidata.org/entity/Q315059'), ('artistName', 'A. E. van Vogt')]
[('artist', 'http://www.wikidata.org/entity/Q243757'), ('artistName', 'A. J. Ayer')]
[('artist', 'http://www.wikidata.org/entity/Q179059'), ('artistName', 'A. J. Cronin')]
[('artist', 'http://www.wikidata.org/entity/Q278872'), ('artistName', 'A. L. Kennedy')]
[('artist', 'http://www.wikidata.org/entity/Q720153'), ('artistName', 'A. Philip Randolph')]
[('artist', 'h

In [6]:
#Now I get only the ones of the movement Renaissance (Q4692), I get the identifier from previous tasks. 
queryString = """

SELECT DISTINCT ?artist ?artistName 
WHERE {

   #bind something 
   ?artist wdt:P31 wd:Q5 . #all instance of human 
   ?artist wdt:P135 wd:Q4692 . #all that has the property movement.   
   # get the label 
   ?artist sc:name ?artistName . 
   
   
}
ORDER BY ASC (?artistName)
LIMIT 20 #I put the limit because there's gonna be a huge amount of people. 



"""

print("Results")
x=run_query(queryString)

Results
[('artist', 'http://www.wikidata.org/entity/Q2782415'), ('artistName', 'Achille Falcone')]
[('artist', 'http://www.wikidata.org/entity/Q1115404'), ('artistName', 'Adolfo de Carolis')]
[('artist', 'http://www.wikidata.org/entity/Q3606113'), ('artistName', 'Aert Mijtens')]
[('artist', 'http://www.wikidata.org/entity/Q395689'), ('artistName', 'Agostino Nifo')]
[('artist', 'http://www.wikidata.org/entity/Q5660492'), ('artistName', 'Agustí Pujol')]
[('artist', 'http://www.wikidata.org/entity/Q5681'), ('artistName', 'Andrea Mantegna')]
[('artist', 'http://www.wikidata.org/entity/Q3615832'), ('artistName', 'Andrea Marone')]
[('artist', 'http://www.wikidata.org/entity/Q283580'), ('artistName', 'Andrea Riccio')]
[('artist', 'http://www.wikidata.org/entity/Q495110'), ('artistName', 'Andrea Solari')]
[('artist', 'http://www.wikidata.org/entity/Q183458'), ('artistName', 'Andrea del Verrocchio')]
[('artist', 'http://www.wikidata.org/entity/Q195655'), ('artistName', 'Andrea della Robbia')]
[

In [7]:
#The next step now that I have all the artists from that period is to get the occupations 
queryString = """

SELECT DISTINCT ?artist ?artistName ?occupationsName 
WHERE {

   #bind something 
   ?artist wdt:P31 wd:Q5 . #all instance of human 
   ?artist wdt:P135 wd:Q4692 . #all that has the property movement. 
   ?artist wdt:P106 ?occupation . 
   
   # get the label 
   ?artist sc:name ?artistName . 
   ?occupation sc:name ?occupationsName .
   
}
ORDER BY ASC (?artistName)
LIMIT 20 #I put the limit because there's gonna be a huge amount of people. 



"""

print("Results")
x=run_query(queryString)

Results
[('artist', 'http://www.wikidata.org/entity/Q2782415'), ('artistName', 'Achille Falcone'), ('occupationsName', 'composer')]
[('artist', 'http://www.wikidata.org/entity/Q1115404'), ('artistName', 'Adolfo de Carolis'), ('occupationsName', 'painter')]
[('artist', 'http://www.wikidata.org/entity/Q1115404'), ('artistName', 'Adolfo de Carolis'), ('occupationsName', 'engraver')]
[('artist', 'http://www.wikidata.org/entity/Q1115404'), ('artistName', 'Adolfo de Carolis'), ('occupationsName', 'photographer')]
[('artist', 'http://www.wikidata.org/entity/Q1115404'), ('artistName', 'Adolfo de Carolis'), ('occupationsName', 'illustrator')]
[('artist', 'http://www.wikidata.org/entity/Q1115404'), ('artistName', 'Adolfo de Carolis'), ('occupationsName', 'fashion figure artist')]
[('artist', 'http://www.wikidata.org/entity/Q3606113'), ('artistName', 'Aert Mijtens'), ('occupationsName', 'painter')]
[('artist', 'http://www.wikidata.org/entity/Q395689'), ('artistName', 'Agostino Nifo'), ('occupatio

On the first 20 people we can already see all the professions that we were looking for, so lets resume that. 

In [9]:
#Now I change the query to only get the occupation and that each ocupation only will be show one time. 
queryString = """

SELECT DISTINCT ?occupation ?occupationsName 
WHERE {

   #bind something 
   ?artist wdt:P31 wd:Q5 . #all instance of human 
   ?artist wdt:P135 wd:Q4692 . #all that has the property movement. 
   ?artist wdt:P106 ?occupation . #the value from occupation on the variable ?occupation 
   
   # get the label 
   
   ?occupation sc:name ?occupationsName .
   
}
ORDER BY ASC (?occupationsName)
LIMIT 200



"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q90878511'), ('occupationsName', 'Carmelite friar')]
[('occupation', 'http://www.wikidata.org/entity/Q250867'), ('occupationsName', 'Catholic priest')]
[('occupation', 'http://www.wikidata.org/entity/Q1636680'), ('occupationsName', 'Renaissance humanist')]
[('occupation', 'http://www.wikidata.org/entity/Q21550346'), ('occupationsName', 'aesthetician')]
[('occupation', 'http://www.wikidata.org/entity/Q42973'), ('occupationsName', 'architect')]
[('occupation', 'http://www.wikidata.org/entity/Q1792450'), ('occupationsName', 'art historian')]
[('occupation', 'http://www.wikidata.org/entity/Q17391638'), ('occupationsName', 'art theorist')]
[('occupation', 'http://www.wikidata.org/entity/Q1294787'), ('occupationsName', 'artisan')]
[('occupation', 'http://www.wikidata.org/entity/Q483501'), ('occupationsName', 'artist')]
[('occupation', 'http://www.wikidata.org/entity/Q155647'), ('occupationsName', 'astrologer')]
[('occupation', 'http://w

We've got all the occupations from that period and we found there the ones that we've been looking for, so we filter to get only that ones

Final query for this task

In [14]:
#Now we change the last query to show only composer (Q36834), painter (Q1028181), sculpor (Q1281618) and poet (Q49757) . 
queryString = """

SELECT DISTINCT ?occupation ?occupationsName 
WHERE {

   #bind something 
   ?artist wdt:P31 wd:Q5 . #all instance of human 
   ?artist wdt:P135 wd:Q4692 . #all that has the property movement. 
   ?artist wdt:P106 ?occupation . 
   
   VALUES ?occupation {wd:Q36834 wd:Q1028181 wd:Q1281618 wd:Q49757 } #This limit the values for ?occupation to only the ones between the brackets. 
   
   # get the label 
   
   ?occupation sc:name ?occupationsName .
   
}
ORDER BY ASC (?occupationsName)
LIMIT 200



"""

print("Results")
x=run_query(queryString)


Results
[('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationsName', 'composer')]
[('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationsName', 'painter')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('occupationsName', 'poet')]
[('occupation', 'http://www.wikidata.org/entity/Q1281618'), ('occupationsName', 'sculptor')]
4


## Task 4
Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

In [47]:
#First I get all the humans that are painters. 

queryString = """

SELECT DISTINCT ?painters ?paintersName 
WHERE {

   # bind something
   ?painters wdt:P31 wd:Q5 . #I filter all the humans in the variable ?painters
   ?painters wdt:P106 wd:Q1028181 . #now I add the condition to the occupation has to be "painters"
   
   
   # get the label 
   ?painters sc:name ?paintersName . 
   
}
ORDER BY ?paintersName

LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('painters', 'http://www.wikidata.org/entity/Q21647924'), ('paintersName', '"Tattoo Jack" Cripe')]
[('painters', 'http://www.wikidata.org/entity/Q98971886'), ('paintersName', "'Abbas Kowsari")]
[('painters', 'http://www.wikidata.org/entity/Q12817104'), ('paintersName', "'Abd Allah Musawwir")]
[('painters', 'http://www.wikidata.org/entity/Q13422938'), ('paintersName', "'Abd al-Hayy")]
[('painters', 'http://www.wikidata.org/entity/Q28465157'), ('paintersName', "'the other' Jan van Kessel")]
[('painters', 'http://www.wikidata.org/entity/Q21452732'), ('paintersName', '(Agnes) Mabel Mackinlay')]
[('painters', 'http://www.wikidata.org/entity/Q59369592'), ('paintersName', '(Américo ?) Aragundo (Aragunde)')]
[('painters', 'http://www.wikidata.org/entity/Q18608042'), ('paintersName', '(B.?) van Kessel')]
[('painters', 'http://www.wikidata.org/entity/Q28720006'), ('paintersName', '(C. or O. ) Williams')]
[('painters', 'http://www.wikidata.org/entity/Q41784927'), ('paintersName', '(Rober

In [50]:
#Now I get the painters but with a diferent form and showing the occupation to prepare the query to add the rest of occupations. 

queryString = """

SELECT DISTINCT ?humans ?humansName ?occupation ?occupationName
WHERE {

   #bind something 
   
   ?humans wdt:P31 wd:Q5 . 
   ?humans wdt:P106 ?occupation. #I get all the occupations of the humans on the variable ?occupation 
                        #painters
   VALUES ?occupation { wd:Q1028181 } #With values I stablish the value for occupation to painters. 
   
   
   # get the label 
   ?humans sc:name ?humansName .
   ?occupation sc:name ?occupationName
   
}
ORDER BY ?humansName

LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('humans', 'http://www.wikidata.org/entity/Q21647924'), ('humansName', '"Tattoo Jack" Cripe'), ('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter')]
[('humans', 'http://www.wikidata.org/entity/Q98971886'), ('humansName', "'Abbas Kowsari"), ('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter')]
[('humans', 'http://www.wikidata.org/entity/Q12817104'), ('humansName', "'Abd Allah Musawwir"), ('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter')]
[('humans', 'http://www.wikidata.org/entity/Q13422938'), ('humansName', "'Abd al-Hayy"), ('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter')]
[('humans', 'http://www.wikidata.org/entity/Q28465157'), ('humansName', "'the other' Jan van Kessel"), ('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter')]
[('humans', 'http://www.wikidata.org/entity/Q21452732'), ('humansNa

In [52]:
#Now I add to the painters, the composers, sculptors and poets. 
queryString = """

SELECT DISTINCT ?humans ?humansName ?occupation ?occupationName 
WHERE {

   #bind something 
   
   ?humans wdt:P31 wd:Q5 .
   ?humans wdt:P106 ?occupation. #I get all the occupations of the humans on the variable ?occupation  
                        #Composer   #painters   #sculptors  #poets 
   VALUES ?occupation { wd:Q36834 wd:Q1028181 wd:Q1281618  wd:Q49757} # With values I stablish the value for occupation to all of the occupations. 
   
   
   # get the label 
   ?humans sc:name ?humansName .
   ?occupation sc:name ?occupationName
   
}
ORDER BY ?occupationName

LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('humans', 'http://www.wikidata.org/entity/Q11686949'), ('humansName', 'José Ximénez'), ('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer')]
[('humans', 'http://www.wikidata.org/entity/Q11871976'), ('humansName', 'Klaus Wirzenius'), ('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer')]
[('humans', 'http://www.wikidata.org/entity/Q240377'), ('humansName', 'Amanda Palmer'), ('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer')]
[('humans', 'http://www.wikidata.org/entity/Q328722'), ('humansName', 'Sebastián Durón'), ('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer')]
[('humans', 'http://www.wikidata.org/entity/Q3430855'), ('humansName', 'Richard Hazard'), ('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer')]
[('humans', 'http://www.wikidata.org/entity/Q8199438'), ('humansName', 'Andrés de Sola'), ('occupa

In [54]:
#Now I count how many of each occupation are. 
queryString = """

SELECT DISTINCT ?occupation ?occupationName (COUNT(*) AS ?people)
WHERE {

   #bind something 
   
   ?humans wdt:P31 wd:Q5 .
   ?humans wdt:P106 ?occupation. #I get all the occupations of the humans on the variable ?occupation 
                        #Composer   #painters   #sculpors  #poets 
   VALUES ?occupation { wd:Q36834 wd:Q1028181 wd:Q1281618  wd:Q49757} # With values I stablish the value for occupation to all of the occupations.
   
   
   # get the label 
   #?humans sc:name ?humansName .
   ?occupation sc:name ?occupationName
   
}
GROUP BY ?occupation ?occupationName
ORDER BY ?occupationName

LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer'), ('people', '90917')]
[('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter'), ('people', '176719')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('occupationName', 'poet'), ('people', '93463')]
[('occupation', 'http://www.wikidata.org/entity/Q1281618'), ('occupationName', 'sculptor'), ('people', '44250')]
4


In [55]:
# I've already have all the people for each occupation, now, I have to filter those who belong to the diferent movements. 
# First I'll try it only with those who belong to Renaissance. 

queryString = """

SELECT DISTINCT ?occupation ?occupationName (COUNT(*) AS ?people)
WHERE {
    
   #bind something 
   
   ?humans wdt:P31 wd:Q5 .
   ?humans wdt:P106 ?occupation. #I get all the occupations of the humans on the variable ?occupation 
                        #Composer   #painters   #sculpors  #poets 
   VALUES ?occupation { wd:Q36834 wd:Q1028181 wd:Q1281618  wd:Q49757} # With values I stablish the value for occupation to all of the occupations.
    
   ?humans wdt:P135 ?movement . #I put the values for movement from everyone to the variable ?movement
                    #renaissance   
   VALUES ?movement { wd:Q4692 } # I use values to filter the variable ?movement and that only gets the ones from Renaissance
   
   # get the label 
   #?humans sc:name ?humansName .
   ?occupation sc:name ?occupationName
   
}
GROUP BY ?occupation ?occupationName
ORDER BY ?occupationName

LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer'), ('people', '8')]
[('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter'), ('people', '55')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('occupationName', 'poet'), ('people', '15')]
[('occupation', 'http://www.wikidata.org/entity/Q1281618'), ('occupationName', 'sculptor'), ('people', '18')]
4


In [56]:
#I add the rest of movements to the filter. 
queryString = """

SELECT DISTINCT ?occupation ?occupationName (COUNT(*) AS ?people)
WHERE {
   
   #bind something 
   
   ?humans wdt:P31 wd:Q5 .
   ?humans wdt:P106 ?occupation. #I get all the occupations of the humans on the variable ?occupation 
                        #Composer   #painters   #sculpors  #poets 
   VALUES ?occupation { wd:Q36834 wd:Q1028181 wd:Q1281618  wd:Q49757} # With values I stablish the value for occupation to all of the occupations.
   
   ?humans wdt:P135 ?movement . #I put the values for movement from everyone to the variable ?movement
                    #renaissance  #Baroque   #Romanticism
   VALUES ?movement { wd:Q4692    wd:Q37853   wd:Q37068} # # I use values to filter the variable ?movement to get all the ones from the three movements
   
   
   # get the label 
   #?humans sc:name ?humansName .
   ?occupation sc:name ?occupationName
   
}
GROUP BY ?occupation ?occupationName
ORDER BY ?occupationName

LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q36834'), ('occupationName', 'composer'), ('people', '27')]
[('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('occupationName', 'painter'), ('people', '585')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('occupationName', 'poet'), ('people', '205')]
[('occupation', 'http://www.wikidata.org/entity/Q1281618'), ('occupationName', 'sculptor'), ('people', '358')]
4


Final query for this task

In [57]:
# write your final query
# Now I show the result separated by movement and occupation. 
#To do that I add to the select the variable ?movementName, I add movement to the group by, and i order by the movement. 
queryString = """

SELECT DISTINCT ?movementName ?occupationName (COUNT(*) AS ?people) 
WHERE {
   
   
   #bind something 
   
   ?humans wdt:P31 wd:Q5 .
   ?humans wdt:P106 ?occupation. #I get all the occupations of the humans on the variable ?occupation 
                        #Composer   #painters   #sculpors  #poets 
   VALUES ?occupation { wd:Q36834 wd:Q1028181 wd:Q1281618  wd:Q49757}  # With values I stablish the value for occupation to all of the occupations.
   
   ?humans wdt:P135 ?movement . #I put the values for movement from everyone to the variable ?movement
                    #renaissance  #Baroque   #Romanticism
   VALUES ?movement { wd:Q4692    wd:Q37853   wd:Q37068} # I use values to filter the variable ?movement to get all the ones from the three movements
   
   # get the label 
   #?humans sc:name ?humansName .
   ?occupation sc:name ?occupationName .
   ?movement sc:name ?movementName. 
}
GROUP BY ?movementName ?occupationName
ORDER BY ?movementName
LIMIT 30


"""

print("Results")
x=run_query(queryString)

Results
[('movementName', 'Baroque'), ('occupationName', 'sculptor'), ('people', '326')]
[('movementName', 'Baroque'), ('occupationName', 'poet'), ('people', '39')]
[('movementName', 'Baroque'), ('occupationName', 'painter'), ('people', '378')]
[('movementName', 'Baroque'), ('occupationName', 'composer'), ('people', '6')]
[('movementName', 'Renaissance'), ('occupationName', 'sculptor'), ('people', '18')]
[('movementName', 'Renaissance'), ('occupationName', 'poet'), ('people', '15')]
[('movementName', 'Renaissance'), ('occupationName', 'composer'), ('people', '8')]
[('movementName', 'Renaissance'), ('occupationName', 'painter'), ('people', '55')]
[('movementName', 'Romanticism'), ('occupationName', 'composer'), ('people', '13')]
[('movementName', 'Romanticism'), ('occupationName', 'painter'), ('people', '152')]
[('movementName', 'Romanticism'), ('occupationName', 'sculptor'), ('people', '14')]
[('movementName', 'Romanticism'), ('occupationName', 'poet'), ('people', '151')]
12


## Task 5
Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

In [3]:
#First I have to find the BGP for sculpture. 
#For that I take a look to the properties of Donatello.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/dir

In [54]:
#For the previous task, I know that donatello is a sculptor. 
#One of his properties is notable work, so I'm going to see what are the values for that propertie. 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   ?obj wdt:P800 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q3853558'), ('iname', 'Medal of the Pazzi conspiracy')]
1


In [9]:
#I get a medal not a sculpture so I explore other paths to get to the sculpture. 
#I'll try with exploring the properties of sculptor. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1281618  ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P6186'), ('name', 'category for eponymous categories')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
10


In [10]:
#The first property for sculptor is product or material produced, I search what's inside that property. 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q1281618 ?p ?obj .
   ?obj wdt:P1056 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q36649'), ('iname', 'visual art')]
[('inst', 'http://www.wikidata.org/entity/Q860861'), ('iname', 'sculpture')]
2


Now that I have the BGP for sculpture, I search for all the subtypes, for that I look for all the things that on the property subclass of has the value for sculpture. 

In [14]:
#I try to get all the subtypes of sculpture. 
queryString = """
SELECT ?subtypes ?subtypesName
WHERE {
   # bind something
   
   ?subtypes wdt:P279 wd:Q860861 . 
   
   # get the label
   ?subtypes sc:name ?subtypesName .
}
ORDER BY ?subtypesName 
"""

print("Results")
x=run_query(queryString)

Results
[('subtypes', 'http://www.wikidata.org/entity/Q106766811'), ('subtypesName', 'A Way of Organising a Pig')]
[('subtypes', 'http://www.wikidata.org/entity/Q402996'), ('subtypesName', 'Ahu in Easter Island')]
[('subtypes', 'http://www.wikidata.org/entity/Q63098560'), ('subtypesName', 'Aphrodite in ancient Greek sculptures')]
[('subtypes', 'http://www.wikidata.org/entity/Q104904962'), ('subtypesName', 'Big thing')]
[('subtypes', 'http://www.wikidata.org/entity/Q43302581'), ('subtypesName', 'Bodhisattva statues of Sri Lanka')]
[('subtypes', 'http://www.wikidata.org/entity/Q5068850'), ('subtypesName', 'Chalkware')]
[('subtypes', 'http://www.wikidata.org/entity/Q5738113'), ('subtypesName', 'Chavin Tenon-heads')]
[('subtypes', 'http://www.wikidata.org/entity/Q282158'), ('subtypesName', 'Cuauhxicalli')]
[('subtypes', 'http://www.wikidata.org/entity/Q637011'), ('subtypesName', 'Cubist sculpture')]
[('subtypes', 'http://www.wikidata.org/entity/Q5228052'), ('subtypesName', 'Daunian stele')

In [45]:
#With the last query that provides all the subtypes from sculpture, now I have to make an upper query to get all the items. 
queryString = """
SELECT ?sculptures ?sculpturesName
WHERE {
    
   # bind something
   
   ?sculptures wdt:P31 ?subtypes. #Now I get all the items that are sculptures or some subtype of sculpture.
   
    
  
  {
        #This gets all the subtypes from sculpture. 
        
        SELECT ?subtypes ?subtypesName
        WHERE {
           # bind something
   
           ?subtypes wdt:P279 wd:Q860861 .
   
           # get the label
           ?subtypes sc:name ?subtypesName .
        }
        ORDER BY ?subtypesName 

  
  
  }
   
   # get the label
   ?sculptures sc:name ?sculpturesName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sculptures', 'http://www.wikidata.org/entity/Q17044462'), ('sculpturesName', "Mamas Don't Let Your Babies Grow Up To Be Cowboys")]
[('sculptures', 'http://www.wikidata.org/entity/Q22248929'), ('sculpturesName', 'Statue of Liberty (Place de la Concorde)')]
[('sculptures', 'http://www.wikidata.org/entity/Q22936897'), ('sculpturesName', 'Heaven of Delight')]
[('sculptures', 'http://www.wikidata.org/entity/Q20741519'), ('sculpturesName', 'Rheinmetall/Victoria 8')]
[('sculptures', 'http://www.wikidata.org/entity/Q27229650'), ('sculpturesName', 'Klausstud')]
[('sculptures', 'http://www.wikidata.org/entity/Q27229991'), ('sculpturesName', 'Gottfried Keller Gedenkstätte')]
[('sculptures', 'http://www.wikidata.org/entity/Q29111872'), ('sculpturesName', 'Saint sépulcre de Marienthal')]
[('sculptures', 'http://www.wikidata.org/entity/Q1922245'), ('sculpturesName', 'Merzbau')]
[('sculptures', 'http://www.wikidata.org/entity/Q20741397'), ('sculpturesName', 'VALIE EXPORT SMART EXPORT')]
[(

In [53]:
#Then I have to see the properties from the sculptures to get the property for country to filter those from Italy. 
#I explore statue of liberty Q22248929
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q22248929  ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('name', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('name', 'architect')]
[('p', 'http://www.wikidata.org/prop/direct/P5816'), ('name', 'state of conservation')]
10


In [62]:
#Now I need the identifier for Italy. 
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   ?obj wdt:P19 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q2044'), ('iname', 'Florence')]
[('inst', 'http://www.wikidata.org/entity/Q2044'), ('iname', 'Florence')]
[('inst', 'http://www.wikidata.org/entity/Q2044'), ('iname', 'Florence')]
3


In [63]:
#Now the properties for Florence 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2044 ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('name', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('name', 'capital of')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('name', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('name', 'category for people who died here')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('

In [64]:
#Now the value on country P17.
queryString = """
SELECT ?inst ?iname 
WHERE {
   # bind something
   wd:Q2044 ?p ?obj .
   ?obj wdt:P17 ?inst.
   
   # get the label
   ?inst sc:name ?iname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('inst', 'http://www.wikidata.org/entity/Q1028'), ('iname', 'Morocco')]
[('inst', 'http://www.wikidata.org/entity/Q142'), ('iname', 'France')]
[('inst', 'http://www.wikidata.org/entity/Q142'), ('iname', 'France')]
[('inst', 'http://www.wikidata.org/entity/Q145'), ('iname', 'United Kingdom')]
[('inst', 'http://www.wikidata.org/entity/Q148'), ('iname', "People's Republic of China")]
[('inst', 'http://www.wikidata.org/entity/Q148540'), ('iname', 'Republic of Florence')]
[('inst', 'http://www.wikidata.org/entity/Q155'), ('iname', 'Brazil')]
[('inst', 'http://www.wikidata.org/entity/Q17'), ('iname', 'Japan')]
[('inst', 'http://www.wikidata.org/entity/Q17'), ('iname', 'Japan')]
[('inst', 'http://www.wikidata.org/entity/Q172579'), ('iname', 'Kingdom of Italy')]
[('inst', 'http://www.wikidata.org/entity/Q172579'), ('iname', 'Kingdom of Italy')]
[('inst', 'http://www.wikidata.org/entity/Q183'), ('iname', 'Germany')]
[('inst', 'http://www.wikidata.org/entity/Q183'), ('iname', 'Germany')

In [66]:
#The property Country is P17 so now I get only the ones in Italy Q38. 
queryString = """
SELECT ?sculptures ?sculpturesName ?countryName
WHERE {
    
   # bind something
   
   ?sculptures wdt:P31 ?subtypes. #Now I get all the items that are sculptures or some subtype of sculpture.
   ?sculptures wdt:P17 wd:Q38 . #I get the country to be Italy.
   ?sculptures 
  
  {
        #This gets all the subtypes from sculpture. 
        
        SELECT ?subtypes ?subtypesName
        WHERE {
           # bind something
   
           ?subtypes wdt:P279 wd:Q860861 .
   
           # get the label
           ?subtypes sc:name ?subtypesName .
        }
        ORDER BY ?subtypesName 
  
  }
   
   # get the label
   ?sculptures sc:name ?sculpturesName.
   #?country sc:name ?countryName .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sculptures', 'http://www.wikidata.org/entity/Q13431140'), ('sculpturesName', 'Panel relief showing Marcus Aurelius performing a sacrifice')]
[('sculptures', 'http://www.wikidata.org/entity/Q23301804'), ('sculpturesName', 'Il contrabbandiere')]
[('sculptures', 'http://www.wikidata.org/entity/Q3983835'), ('sculpturesName', 'Teodelapio')]
[('sculptures', 'http://www.wikidata.org/entity/Q24686949'), ('sculpturesName', 'The Floating Piers')]
[('sculptures', 'http://www.wikidata.org/entity/Q59192597'), ('sculpturesName', 'Apotheosis of Sabina')]
[('sculptures', 'http://www.wikidata.org/entity/Q56605032'), ('sculpturesName', 'Sette rilievi in via Terraggio')]
[('sculptures', 'http://www.wikidata.org/entity/Q107091012'), ('sculpturesName', 'Buddha in contemplation (Garau, Milan)')]
[('sculptures', 'http://www.wikidata.org/entity/Q3786839'), ('sculpturesName', 'Hortus Conclusus')]
[('sculptures', 'http://www.wikidata.org/entity/Q66823293'), ('sculpturesName', 'Cercivento: una Bibbia 

In [72]:
#Now I try to limit the property to only the ones in the movement renaissance Q4692. To do that the only idea that occurs to me is to find it through 
#the property creator, and then see to what movement the creator of the sculptor belongs. 
queryString = """
SELECT  ?creator ?creatorName 
WHERE {
    
   # bind something
   
   ?sculptures wdt:P31 ?subtypes. #Now I get all the items that are sculptures or some subtype of sculpture.
   ?sculptures wdt:P17 wd:Q38 . #I get the country to be Italy.
   ?sculptures wdt:P170 ?creator . #I get creator  
   ?creator wdt:P135 wd:Q4692 .

  {
        #This gets all the subtypes from sculpture. 
        
        SELECT ?subtypes ?subtypesName
        WHERE {
           # bind something
   
           ?subtypes wdt:P279 wd:Q860861 .
   
           # get the label
           ?subtypes sc:name ?subtypesName .
        }
        ORDER BY ?subtypesName 
  
  }
   
   # get the label
   ?sculptures sc:name ?sculpturesName.
   ?creator sc:name ?creatorName .
}
ORDER BY ?creatorName 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('creator', 'http://www.wikidata.org/entity/Q183458'), ('creatorName', 'Andrea del Verrocchio')]
[('creator', 'http://www.wikidata.org/entity/Q183458'), ('creatorName', 'Andrea del Verrocchio')]
[('creator', 'http://www.wikidata.org/entity/Q195655'), ('creatorName', 'Andrea della Robbia')]
[('creator', 'http://www.wikidata.org/entity/Q314797'), ('creatorName', 'Jacopo della Quercia')]
[('creator', 'http://www.wikidata.org/entity/Q314797'), ('creatorName', 'Jacopo della Quercia')]
[('creator', 'http://www.wikidata.org/entity/Q861385'), ('creatorName', 'Pellegrino Tibaldi')]
6


In [80]:
#Now I add the movements Baroque an Romanticism. 
queryString = """
SELECT  ?creator ?creatorName ?movementName
WHERE {
    
   # bind something
   
   ?sculptures wdt:P31 ?subtypes. #Now I get all the items that are sculptures or some subtype of sculpture.
   ?sculptures wdt:P17 wd:Q38 . #I get the country to be Italy.
   ?sculptures wdt:P170 ?creator . #I get creator  
   ?creator wdt:P135 ?movement . 
   
        #renaissance  #Baroque   #Romanticism
   VALUES ?movement { wd:Q4692    wd:Q37853   wd:Q37068} # # I use values to filter the variable ?movement to get all the ones from the three movements

  {
        #This gets all the subtypes from sculpture. 
        
        SELECT ?subtypes ?subtypesName
        WHERE {
           # bind something
   
           ?subtypes wdt:P279 wd:Q860861 .
   
           # get the label
           ?subtypes sc:name ?subtypesName .
        }
        ORDER BY ?subtypesName 
  
  }
   
   # get the label
   ?creator sc:name ?creatorName .
   ?movement sc:name ?movementName .
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('creator', 'http://www.wikidata.org/entity/Q122328'), ('creatorName', 'Carlo Maderno'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q122328'), ('creatorName', 'Carlo Maderno'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q122328'), ('creatorName', 'Carlo Maderno'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q160538'), ('creatorName', 'Gian Lorenzo Bernini'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q160538'), ('creatorName', 'Gian Lorenzo Bernini'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q160538'), ('creatorName', 'Gian Lorenzo Bernini'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q160538'), ('creatorName', 'Gian Lorenzo Bernini'), ('movementName', 'Baroque')]
[('creator', 'http://www.wikidata.org/entity/Q160538'), ('creatorName', 'Gian Lorenzo Bernini'), ('movementName', 'Baroque')]
[('cr

Final query for this task

In [81]:
#The last thing to do is count the sculptures and do the 
queryString = """
SELECT  ?movement ?movementName (COUNT(*) AS ?sculptures)
WHERE {
    
   # bind something
   
   ?sculptures wdt:P31 ?subtypes. #Now I get all the items that are sculptures or some subtype of sculpture.
   ?sculptures wdt:P17 wd:Q38 . #I get the country to be Italy.
   ?sculptures wdt:P170 ?creator . #I get creator  
   ?creator wdt:P135 ?movement . 
   
        #renaissance  #Baroque   #Romanticism
   VALUES ?movement { wd:Q4692    wd:Q37853   wd:Q37068} # # I use values to filter the variable ?movement to get all the ones from the three movements

  {
        #This gets all the subtypes from sculpture. 
        
        SELECT ?subtypes ?subtypesName
        WHERE {
           # bind something
   
           ?subtypes wdt:P279 wd:Q860861 .
   
           # get the label
           ?subtypes sc:name ?subtypesName .
        }
        ORDER BY ?subtypesName 
  
  }
   
   # get the label
  
   ?movement sc:name ?movementName .
}
GROUP BY ?movement ?movementName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('movementName', 'Renaissance'), ('sculptures', '6')]
[('movement', 'http://www.wikidata.org/entity/Q37853'), ('movementName', 'Baroque'), ('sculptures', '15')]
[('movement', 'http://www.wikidata.org/entity/Q37068'), ('movementName', 'Romanticism'), ('sculptures', '1')]
3


NOTE: Probably there are only a few because I'm only considering only the movements and not all the variances.

## Task 6
For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

In [86]:
#I obtained the identifier for Italy so I'm going to look for his properties. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
ORDER BY DESC (?name)
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8432'), ('name', 'Österreichisches Musiklexikon Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9318'), ('name', 'Ávvir topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2436'), ('name', 'voltage')]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P4841'), ('name', 'total fertility rate')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P78'), ('name', 'top-level Internet domain')]
[('p', 'http://www.wikidata.org/prop/direct/P3984'), ('name', 'subreddit')]
[('p', 'http://www.wikidata.org/prop/direct/P47'), ('name', 'shares border with')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('name', 'shape')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('n

In [87]:
#I find one called continen P30 so we take a look at its value
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q38 wdt:P30 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q46'), ('oName', 'Europe')]
1


In [105]:
#Let's start to find all the poets form the romanticism
#I use the properties movement, and romanticism form previous queries. 
queryString = """

SELECT DISTINCT ?poets ?poetsName ?romanticsName
WHERE {

   # bind something
   ?poets wdt:P31 wd:Q5 . #I filter all the humans in the variable ?poets
   ?poets wdt:P106 wd:Q49757 . # I make that the occupation has to be poet
   ?poets wdt:P135 ?romantics. #I put on romantics the value for movement. 
   
   VALUES ?romantics { wd:Q37068 } #The value for romantics has to be romanticism. 
   
   # get the label 
   ?poets sc:name ?poetsName . 
   ?romantics sc:name ?romanticsName .
}
ORDER BY DESC (?romanticsName)

LIMIT 10


"""

print("Results")
x=run_query(queryString)

Results
[('poets', 'http://www.wikidata.org/entity/Q5932129'), ('poetsName', 'Joaquín José Cervino'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q5258787'), ('poetsName', 'Tobias Barreto'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q5340521'), ('poetsName', 'Eduardo Asquerino'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q573223'), ('poetsName', 'Patricio de la Escosura'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q688529'), ('poetsName', 'Johannes Carsten Hauch'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q597298'), ('poetsName', 'Antoni Edward Odyniec'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q645491'), ('poetsName', 'Yevhen Hrebinka'), ('romanticsName', 'Romanticism')]
[('poets', 'http://www.wikidata.org/entity/Q93343'), ('poetsName', 'Percy Bysshe Shelley'), ('romanticsName'

In [115]:
#Now the next step is to take the country from every poet and it has to be on Europe. 
queryString = """

SELECT DISTINCT ?poets ?poetsName ?country ?countryName
WHERE {

   # bind something
   ?poets wdt:P31 wd:Q5 . #I filter all the humans in the variable ?poets
   ?poets wdt:P106 wd:Q49757 . # I make that the occupation has to be poet
   ?poets wdt:P27 ?country. 
   ?country wdt:P30 wd:Q46 .
   
   ?poets wdt:P135 ?romantics. #I put on romantics the value for movement. 
   
   VALUES ?romantics { wd:Q37068 } #The value for romantics has to be romanticism. 
   
   
   # get the label 
   ?poets sc:name ?poetsName .
   ?country sc:name ?countryName .
}
ORDER BY ?countryName
LIMIT 20



"""

print("Results")
x=run_query(queryString)

Results
[('poets', 'http://www.wikidata.org/entity/Q352914'), ('poetsName', 'Karel Jaromír Erben'), ('country', 'http://www.wikidata.org/entity/Q28513'), ('countryName', 'Austria-Hungary')]
[('poets', 'http://www.wikidata.org/entity/Q3498826'), ('poetsName', 'Stevan Kaćanski'), ('country', 'http://www.wikidata.org/entity/Q28513'), ('countryName', 'Austria-Hungary')]
[('poets', 'http://www.wikidata.org/entity/Q362089'), ('poetsName', 'Jovan Jovanović Zmaj'), ('country', 'http://www.wikidata.org/entity/Q28513'), ('countryName', 'Austria-Hungary')]
[('poets', 'http://www.wikidata.org/entity/Q352914'), ('poetsName', 'Karel Jaromír Erben'), ('country', 'http://www.wikidata.org/entity/Q131964'), ('countryName', 'Austrian Empire')]
[('poets', 'http://www.wikidata.org/entity/Q1031'), ('poetsName', 'France Prešeren'), ('country', 'http://www.wikidata.org/entity/Q131964'), ('countryName', 'Austrian Empire')]
[('poets', 'http://www.wikidata.org/entity/Q938402'), ('poetsName', 'Laza Kostić'), ('co

Final query for this task

In [117]:
#Now we don't show the poet and count all the poets. 
queryString = """

SELECT DISTINCT ?country ?countryName (COUNT(*) AS ?poets)
WHERE {

   # bind something
   ?poets wdt:P31 wd:Q5 . #I filter all the humans in the variable ?poets
   ?poets wdt:P106 wd:Q49757 . # I make that the occupation has to be poet
   ?poets wdt:P27 ?country. 
   ?country wdt:P30 wd:Q46 .
   
   ?poets wdt:P135 ?romantics. #I put on romantics the value for movement. 
   
   VALUES ?romantics { wd:Q37068 } #The value for romantics has to be romanticism. 
   
   
   # get the label 
   ?poets sc:name ?poetsName .
   ?country sc:name ?countryName .
}
ORDER BY ?countryName
LIMIT 100



"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q28513'), ('countryName', 'Austria-Hungary'), ('poets', '3')]
[('country', 'http://www.wikidata.org/entity/Q131964'), ('countryName', 'Austrian Empire'), ('poets', '6')]
[('country', 'http://www.wikidata.org/entity/Q227'), ('countryName', 'Azerbaijan'), ('poets', '1')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('poets', '3')]
[('country', 'http://www.wikidata.org/entity/Q221457'), ('countryName', 'Congress Poland'), ('poets', '1')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('poets', '4')]
[('country', 'http://www.wikidata.org/entity/Q156199'), ('countryName', 'Electorate of Saxony'), ('poets', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('poets', '16')]
[('country', 'http://www.wikidata.org/entity/Q151624'), ('countryName', 'German Confederation'), ('poets', '1')]
[('country', 'http://www.wikidata.org/entity/Q183'), (

## Task 7
Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

In [120]:
#lets start to find all the sculptures 
queryString = """
SELECT ?sculptures ?sculpturesName
WHERE {

   # bind something
   
   ?sculptures wdt:P31 wd:Q860861 .
   
   
   # get the label
   ?sculptures sc:name ?sculpturesName .
}
ORDER BY ?sculpturesName 
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('sculptures', 'http://www.wikidata.org/entity/Q63847952'), ('sculpturesName', '"Augustus" (Self-Portrait) [obverse]')]
[('sculptures', 'http://www.wikidata.org/entity/Q96238003'), ('sculpturesName', '"Beauty" and "Work"')]
[('sculptures', 'http://www.wikidata.org/entity/Q100368647'), ('sculpturesName', '"Bärenbrunnen" von Franz Barwig dem Jüngeren, Dr.-Karl-Landsteiner-Park, Vienna')]
[('sculptures', 'http://www.wikidata.org/entity/Q10655867'), ('sculpturesName', '"Carvings in concrete"')]
[('sculptures', 'http://www.wikidata.org/entity/Q99521237'), ('sculpturesName', '"Denkmal für eine Wirtschaftsnobelpreis-Trägerin" von Helmut und Johanna Kandl, Volkertplatz, Wien Leopoldstadt')]
[('sculptures', 'http://www.wikidata.org/entity/Q99438396'), ('sculpturesName', '"Der Ackerbau", Karlsplatz')]
[('sculptures', 'http://www.wikidata.org/entity/Q99438194'), ('sculpturesName', '"Der letzte Mensch", Karlsplatz')]
[('sculptures', 'http://www.wikidata.org/entity/Q99438386'), ('sculpture

In [121]:
#I'm going to search for the properties of the sculptures to find the properties that says where they are located. 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q63847952  ?p ?obj .
   FILTER(!isLiteral(?obj)).
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6108'), ('name', 'IIIF manifest')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
6


In [130]:
#I find the property location P276, now I modify the previous query to get all the sculptures that are in the louvre museum. 
queryString = """
SELECT ?sculptures ?sculpturesName
WHERE {

   # bind something
   
   ?sculptures wdt:P31 wd:Q860861 .
   ?sculptures wdt:P276 wd:Q19675. 
   
   
   # get the label
   ?sculptures sc:name ?sculpturesName .
}
ORDER BY ?sculpturesName 
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sculptures', 'http://www.wikidata.org/entity/Q36707230'), ('sculpturesName', 'Fountain of Diana')]
[('sculptures', 'http://www.wikidata.org/entity/Q3178029'), ('sculpturesName', 'Jeune fille de Mégare')]
[('sculptures', 'http://www.wikidata.org/entity/Q681067'), ('sculpturesName', 'Lady of Elche')]
[('sculptures', 'http://www.wikidata.org/entity/Q1439502'), ('sculpturesName', 'Napoleon as Mars the Peacemaker')]
[('sculptures', 'http://www.wikidata.org/entity/Q3501426'), ('sculpturesName', 'Stela of Pasenhor')]
5


Final query for this task

In [134]:
#Now We get all the ones for the 3 movements. 
queryString = """
SELECT ?movement ?movementsName (COUNT(*) AS ?scul)
WHERE {

   # bind something
   
   ?sculptures wdt:P31 wd:Q860861 .
   ?sculptures wdt:P276 wd:Q19675. 
   ?sculptures wdt:P135 ?movement .
   
   #renaissance  #Baroque   #Romanticism
   VALUES ?movement { wd:Q4692    wd:Q37853   wd:Q37068} # # I use values to filter the variable ?movement to get all the ones from the three movements
   
   # get the label
   ?movement sc:name ?movementsName .
   
}
GROUP BY ?movement ?movementsName
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
Empty


I don't get any sculpture for anyone of the movements on the louvre museum.